# Using the Energy Efficiency Report Pipeline in FABRIC

IMPORTANT: this is an additional approach that can be taken for the energy efficiency report pipeline. However, see energy-efficiency/README.md for an explanation on a better and recommended approach with an SSH tunnel on your local machine to the Prometheus deployment on the FABRIC node. This way, you can follow the same steps as you would locally, with Prometheus accessible from the local machine at localhost:9090, in this case tunneling to the Prometheus deployment on the FABRIC node.

This is a short notebook that explains how to execute the energy efficiency pipeline created in this project. Make sure to follow this after the steps of the [Experiments](./experiments.ipynb) notebook have been followed, since this notebook reuses some of those things.

1. The only difference is the Prometheus URL that should be queried. So, copy all the files in the energy-efficiency/data-collection folder in the FABRIC Jupyter Hub environment (for example in a separate folder to separate it from the other files). Upload them all in the same folder.

2. Then replace the PROMETHEUS_URL in the constants.py file in energy-efficiency/data-collection folder with the PROMETHEUS_URL from the constants.py file in fabric/experiments (this folder). This ensures the correct Prometheus URL is used to collect the data. For example:

PROMETHEUS_URL = 'http://10.145.1.5:32471'

Note: no need to port-forward Prometheus on the node, this uses the NodePort that Prometheus is exposed on directly. If a port-forward is executed (kubectl port-forward svc/prometheus-kube-prometheus-prometheus -n monitoring 9090:9090), it will still work, however, it is not necessary in this case.

3. Execute the below code to run the data collection.

4. Finally, place the collected data file in the energy-efficiency/data-collection/data folder and execute the data-analysis file as normal on your local machine. This is because the only difference is the data collection, once the data is collected, it can be analyzed on the local machine without needing FABRIC. See the README.md in the energy-efficiency folder for instructions on how to execute the data analysis.

In [ ]:
# ======================================== Import FABRIC library and other required libraries ======================================== 
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

# ======================================== Set variables ======================================== 
slice_name = 'DYNAMOS_EnergyEfficiency'
# Nodes:
node1_name = 'k8s-control-plane'
node2_name = 'dynamos-core'
node3_name = 'vu'
node4_name = 'uva'
node5_name = 'surf'

# ======================================== Save experiments data ======================================== 
# This separate cell can be used to extract the file 
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Get the correct node to run the experiment on (must be the same node as the experiments.ipynb notebook)
    node = slice.get_node(name=node1_name)

    # Make sure the directory exists:
    node.execute("mkdir -p ~/energy-pipeline")

    # Upload required files for the experiments:
    node.upload_file(local_file_path="constants.py", remote_file_path="./energy-pipeline/constants.py")
    node.upload_file(local_file_path="get_metrics.py", remote_file_path="./energy-pipeline/get_metrics.py")
    node.upload_file(local_file_path="query_prometheus.py", remote_file_path="./energy-pipeline/query_prometheus.py")
    node.upload_file(local_file_path="utils.py", remote_file_path="./energy-pipeline/utils.py")
    
    # Run the experiment. This needs to run the python script to allow the output to be added in the notebook output, with a separate script that did not happen
    # Reuse the virtual python environment created in the experiments.ipynb
    stdout, stderr = node.execute(
        (
            # Make the script executable and 
            f"chmod +x ./energy-pipeline/get_metrics.py && "
            # Go to the corresponding location for the venv
            f"cd ~/experiments && "
            # Activate the venv
            f"source dynamos-env/bin/activate && "
            # Install additional requirements
            f"pip install pandas && "
            # Then go to the location where the data-collection files are
            f"cd ~/energy-pipeline && "
            # Execute the actual experiments. Use -u to use unbuffered mode for stdout, stderr, and stdin, 
            # so that print() calls and logs from inside the python script appear live (or at least flush immediately after each action)
            f"python3 -u get_metrics.py"
        )
    )

    # Download the data file from the node to the Jupyter Notebook environment.
    # This will now save it to the current path, from which you can manually download it and add it in the project.
    # https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.download_file
    node.download_file(remote_file_path="./energy-pipeline/data/energy_metrics.csv", local_file_path="./energy_metrics.csv")

except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()